<small>
Copyright (c) 2017 Andrew Glassner

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.
</small>



# Deep Learning From Basics to Practice
## by Andrew Glassner, https://dlbasics.com, http://glassner.com
------
## Chapter 21: Convolutional Neural Networks (CNNs)
### Notebook 4: Filter activations in VGG16

This notebook is provided as a “behind-the-scenes” look at code used to make some of the figures in this chapter. It is still in the hacked-together form used to develop the figures, and is only lightly commented.

This code is adapted from https://github.com/fchollet/deep-learning-with-python-notebooks

In [ ]:
import keras
from keras.applications import VGG16
from keras import backend as K_backend
import numpy as np
import math
import matplotlib.pyplot as plt

# Just in case the Keras defaults aren't as we expect
K_backend.set_image_data_format('channels_last')

In [ ]:
# Make a File_Helper for saving and loading files.

save_files = True

import os, sys, inspect
current_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
sys.path.insert(0, os.path.dirname(current_dir)) # path to parent dir
from DLBasics_Utilities import File_Helper
file_helper = File_Helper(save_files)

In [ ]:
def get_VGG16():
    model = VGG16(weights='imagenet', include_top=False)
    return model

In [ ]:
def prepare_image_for_display(image):
    # normalize tensor: center on 0., ensure std is 0.1
    image -= image.mean()
    image /= (image.std() + 1e-5)
    image *= 0.1
    image += 0.5

    # convert to RGB array
    image *= 255
    image = np.clip(image, 0, 255).astype('uint8')
    return image

In [ ]:
# This is where the Keras magic happens.
# From https://github.com/fchollet/deep-learning-with-python-notebooks
def get_filter_image(model, layer_name, filter_index, size=150, num_steps=40):
    # Build a loss function that maximizes the activation
    # of the nth filter of the layer considered. We find the
    # average value of all the activations coming out of the filter.
    layer_output = model.get_layer(layer_name).output
    loss = K_backend.mean(layer_output[:, :, :, filter_index])

    # Compute the gradient of the input picture wrt this loss
    grads = K_backend.gradients(loss, model.input)[0]

    # Normalization trick: we normalize the gradient
    grads /= (K_backend.sqrt(K_backend.mean(K_backend.square(grads))) + 1e-5)

    # This function returns the loss and grads given the input picture
    iterate = K_backend.function([model.input], [loss, grads])
    
    # We start from a gray image with some noise
    input_img_data = np.random.random((1, size, size, 3)) * 20 + 128.

    # Run gradient ascent 
    step_size = 1.
    for i in range(num_steps):
        loss_value, grads_value = iterate([input_img_data])
        input_img_data += grads_value * step_size
        
    img = input_img_data[0]
    return prepare_image_for_display(img)

In [ ]:
def show_one_filter(model, layer_name, filter_number, size=150, filename=None):
    filter_image = get_filter_image(model, layer_name, filter_number, size)
    plt.imshow(filter_image)
    plt.xticks([],[])
    plt.yticks([],[])
    plt.title(layer_name)
    file_helper.save_figure(filename)
    plt.show()

In [ ]:
# Show a bunch of filters from the given layer
def show_filter_grid(model, layer_name, filters_list, num_rows, num_cols, filename=None):
    img_size = 64
    gap_size = 3
    
    pixels_wide = (num_cols * img_size) + ((num_cols-1) * gap_size)
    pixels_high = (num_rows * img_size) + ((num_rows-1) * gap_size)

    grid = np.zeros((pixels_high, pixels_wide, 3))
    
    for img_num, filter_number in enumerate(filters_list):
        filter_image = get_filter_image(model, layer_name, filter_number, size=img_size)
        y = img_num // num_cols
        x = img_num - (y*num_cols)
        h_start = x * (img_size + gap_size)
        v_start = y * (img_size + gap_size)
        h_end = h_start + img_size
        v_end = v_start + img_size
        #print("img_num=",img_num," filter_number=",filter_number," y=",y," x=",x)
        grid[v_start:v_end, h_start:h_end, :] = filter_image
    
    plt.figure(figsize=(20, 20))
    plt.imshow(grid)
    plt.title('Filters from VGG16 layer '+layer_name, fontsize=28, y=1.01)
    plt.xticks([],[])
    plt.yticks([],[])
    file_helper.save_figure(filename)
    plt.show()

In [ ]:
# almost identical to show_filter_grid, but shows only selected layer/filter pairs
def show_selections_grid(model, layer_filter_list, num_rows, num_cols, filename=None):
    img_size = 150
    gap_size = 3
    
    pixels_wide = (num_cols * img_size) + ((num_cols-1) * gap_size)
    pixels_high = (num_rows * img_size) + ((num_rows-1) * gap_size)

    grid = np.zeros((pixels_high, pixels_wide, 3))
    
    for img_num, filter_pair in enumerate(layer_filter_list):
        layer_name = filter_pair[0]
        filter_number = filter_pair[1]
        filter_image = get_filter_image(model, layer_name, filter_number, size=img_size)
        y = img_num // num_cols
        x = img_num - (y*num_cols)
        h_start = x * (img_size + gap_size)
        v_start = y * (img_size + gap_size)
        h_end = h_start + img_size
        v_end = v_start + img_size
        #print("img_num=",img_num," filter_number=",filter_number," y=",y," x=",x)
        grid[v_start:v_end, h_start:h_end, :] = filter_image
    
    plt.figure(figsize=(20, 20))
    plt.imshow(grid)
    plt.title('Selected filters from VGG16', fontsize=28, y=1.01)
    plt.xticks([],[])
    plt.yticks([],[])
    file_helper.save_figure(filename)
    plt.show()

In [ ]:
model = get_VGG16()
# Let's see what the layers are named, so we can refer to them
model.summary()

In [ ]:
show_one_filter(model, 'block1_conv1', 0, size=150, filename='block1_conv1-filter-0')

In [ ]:
# Since grids bigger than 8 by 8 get too small for detail, we 
# only show the first 64 filters from each layer.
#
# Structure of the lists:
# block number, conv number, #filters to show, num_rows, num_cols

filter_sets = [
    [1, 1,  64,  8,  8 ],
    [1, 2,  64,  8,  8 ],
    [2, 1,  64,  8,  8 ],
    [2, 2,  64,  8,  8 ],
    [3, 1,  64,  8,  8 ],
    [3, 2,  64,  8,  8 ],
    [3, 3,  64,  8,  8 ],
    [4, 1,  64,  8,  8 ],
    [4, 2,  64,  8,  8 ],
    [4, 3,  64,  8,  8 ],
    [5, 1,  64,  8,  8 ],
    [5, 2,  64,  8,  8 ],
    [5, 3,  64,  8,  8 ],
]

for set in filter_sets:
    block_num, conv_num, num_filters, num_rows, num_cols = set
    layer_name = 'block'+str(block_num)+'_conv'+str(conv_num)
    file_name = 'block'+str(block_num)+'-filter'+str(conv_num)+'-size-'+str(num_filters)
    #print("layer_name=",layer_name," file_name=",file_name)
    show_filter_grid(model, layer_name, range(num_filters), num_rows, num_cols, file_name)

In [ ]:
layer_filter_list_1 = [
    ['block1_conv1', 21],
    ['block1_conv2', 12],
    ['block2_conv2', 29],
    ['block3_conv1', 17],
    ['block3_conv1', 28],
    ['block3_conv1', 59],
    ['block3_conv2', 5],
    ['block3_conv2', 8],
    ['block3_conv3', 20]
]

show_selections_grid(model, layer_filter_list_1, 3, 3, filename='VGG16-selections-1')

In [ ]:
layer_filter_list_2 = [
    ['block3_conv3', 21],
    ['block4_conv1', 0],
    ['block4_conv1', 17],
    ['block4_conv1', 53],
    ['block4_conv1', 47],
    ['block4_conv2', 27],
    ['block5_conv1', 48],
    ['block5_conv2', 25],
    ['block5_conv2', 57]
]

show_selections_grid(model, layer_filter_list_2, 3, 3, filename='VGG16-selections-2')